In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]

    elif is_train==False:
        
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :]

In [3]:
train = pd.read_csv('./data/train/train.csv')

test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    test.append(temp)

df_test = pd.concat(test)

submission = pd.read_csv('./data/sample_submission.csv')

In [4]:
df_train = preprocess_data(train)

df_train.shape, df_test.shape

((52464, 9), (3888, 7))

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train[['Hour','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_train[['Hour','DHI','DNI','WS','RH','T']])
df_test[['Hour','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_test[['Hour','DHI','DNI','WS','RH','T']])

In [6]:
Day  = df_train.iloc[:, :-2]
Day7 = df_train.iloc[:, -2]
Day8 = df_train.iloc[:, -1]

In [7]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras.backend import mean, maximum

tf.random.set_seed(42)

In [8]:
def quantile_loss(q, y, pred):
    err = (y-pred)
    return mean(maximum(q*err, (q-1)*err), axis=-1)

In [9]:
q_lst = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [10]:
from sklearn.model_selection import train_test_split

X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(Day, Day7, test_size=0.25, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(Day, Day8, test_size=0.25, random_state=0)

X_train_1.shape, X_valid_1.shape, Y_train_1.shape, Y_valid_1.shape

((39348, 7), (13116, 7), (39348,), (13116,))

In [11]:
model7 = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(Day.shape)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model8 = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(Day.shape)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

In [12]:
model7.compile(loss='mse', optimizer='adam')
model7.fit(X_train_1, Y_train_1, epochs=25, batch_size=48, validation_split=0.25)
model7.evaluate(X_valid_1, Y_valid_1)

Epoch 1/25
615/615 [==============================] - 1s 2ms/step - loss: 277.3139 - val_loss: 154.7046
Epoch 2/25
615/615 [==============================] - 1s 1ms/step - loss: 140.9160 - val_loss: 145.1606
Epoch 3/25
615/615 [==============================] - 1s 1ms/step - loss: 136.9565 - val_loss: 141.0216
Epoch 4/25
615/615 [==============================] - 1s 1ms/step - loss: 132.5962 - val_loss: 146.5380
Epoch 5/25
615/615 [==============================] - 1s 1ms/step - loss: 132.8096 - val_loss: 139.0438
Epoch 6/25
615/615 [==============================] - 1s 1ms/step - loss: 131.1158 - val_loss: 139.1942
Epoch 7/25
615/615 [==============================] - 1s 1ms/step - loss: 130.3700 - val_loss: 136.1147
Epoch 8/25
615/615 [==============================] - 1s 1ms/step - loss: 130.3074 - val_loss: 135.2806
Epoch 9/25
615/615 [==============================] - 1s 1ms/step - loss: 126.8996 - val_loss: 141.9392
Epoch 10/25
615/615 [==============================] - 1s 1ms/st

128.03042602539062

In [13]:
model8.compile(loss='mse', optimizer='adam')
model8.fit(X_train_2, Y_train_2, epochs=25, batch_size=48, validation_split=0.25)
model8.evaluate(X_valid_2, Y_valid_2)

Epoch 1/25
615/615 [==============================] - 1s 2ms/step - loss: 373.1551 - val_loss: 163.8447
Epoch 2/25
615/615 [==============================] - 1s 1ms/step - loss: 152.5000 - val_loss: 150.2012
Epoch 3/25
615/615 [==============================] - 1s 1ms/step - loss: 150.9749 - val_loss: 147.0889
Epoch 4/25
615/615 [==============================] - 1s 1ms/step - loss: 146.7108 - val_loss: 147.1102
Epoch 5/25
615/615 [==============================] - 1s 1ms/step - loss: 145.8405 - val_loss: 142.2581
Epoch 6/25
615/615 [==============================] - 1s 1ms/step - loss: 141.5623 - val_loss: 144.4173
Epoch 7/25
615/615 [==============================] - 1s 1ms/step - loss: 142.1645 - val_loss: 144.0595
Epoch 8/25
615/615 [==============================] - 1s 1ms/step - loss: 138.7153 - val_loss: 139.1425
Epoch 9/25
615/615 [==============================] - 1s 1ms/step - loss: 138.9164 - val_loss: 137.1708
Epoch 10/25
615/615 [==============================] - 1s 2ms/st

139.7083282470703

In [12]:
results7 = pd.DataFrame()

for q in q_lst:
    model7.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model7.fit(Day, Day7, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model7.predict(df_test))
    results7 = pd.concat([results7, pred], axis=1)

results7.shape

Epoch 1/25
820/820 [==============================] - 3s 2ms/step - loss: 1.4342 - val_loss: 1.6017
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4142 - val_loss: 1.5955
Epoch 3/25
820/820 [==============================] - 1s 2ms/step - loss: 1.4060 - val_loss: 1.5881
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4015 - val_loss: 1.5808
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4021 - val_loss: 1.5879
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3885 - val_loss: 1.6105
Epoch 7/25
820/820 [==============================] - 1s 2ms/step - loss: 1.3810 - val_loss: 1.5975
Epoch 8/25
820/820 [==============================] - 1s 2ms/step - loss: 1.3911 - val_loss: 1.5634
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3757 - val_loss: 1.5665
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3823 - val_loss: 1.5543

820/820 [==============================] - 2s 2ms/step - loss: 2.5519 - val_loss: 2.9665
Epoch 2/25
820/820 [==============================] - 1s 2ms/step - loss: 2.5226 - val_loss: 2.9690
Epoch 3/25
820/820 [==============================] - 1s 2ms/step - loss: 2.5248 - val_loss: 2.9588
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5005 - val_loss: 2.9657
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5374 - val_loss: 2.9783
Epoch 6/25
820/820 [==============================] - 1s 2ms/step - loss: 2.5284 - val_loss: 2.9701
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5400 - val_loss: 2.9648
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5754 - val_loss: 2.9780
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5144 - val_loss: 2.9558
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5635 - val_loss: 2.9432
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.4413 - val_loss: 2.9622
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4442 - val_loss: 2.9911
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4515 - val_loss: 2.9750
Epoch 6/25
820/820 [==============================] - 1s 2ms/step - loss: 2.4633 - val_loss: 2.9701
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4822 - val_loss: 2.9470
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5219 - val_loss: 2.9727
Epoch 9/25
820/820 [==============================] - 1s 2ms/step - loss: 2.4443 - val_loss: 2.9642
Epoch 10/25
820/820 [==============================] - 1s 2ms/step - loss: 2.5172 - val_loss: 2.9259
Epoch 11/25
820/820 [==============================] - 1s 2ms/step - loss: 2.4064 - val_loss: 2.9621
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4196 - val_loss: 2.9492
Epoch 13

Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7780 - val_loss: 2.1868
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8232 - val_loss: 2.1541
Epoch 11/25
820/820 [==============================] - 1s 2ms/step - loss: 1.7507 - val_loss: 2.1525
Epoch 12/25
820/820 [==============================] - 1s 2ms/step - loss: 1.7655 - val_loss: 2.1638
Epoch 13/25
820/820 [==============================] - 1s 2ms/step - loss: 1.7923 - val_loss: 2.1674
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7784 - val_loss: 2.1879
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7717 - val_loss: 2.2074
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7714 - val_loss: 2.1506
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7933 - val_loss: 2.1578
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7814 - val_loss

820/820 [==============================] - 1s 1ms/step - loss: 0.7175 - val_loss: 0.8879
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7119 - val_loss: 0.9314
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7190 - val_loss: 0.8815
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7184 - val_loss: 0.8664
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7156 - val_loss: 0.8790
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7145 - val_loss: 0.8947
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7234 - val_loss: 0.8698
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7295 - val_loss: 0.8734
Epoch 22/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7145 - val_loss: 0.8611
Epoch 23/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7119 - val_loss: 0.9141
Ep

(3888, 9)

In [13]:
results8 = pd.DataFrame()

for q in q_lst:
    model8.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model8.fit(Day, Day8, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model8.predict(df_test))
    results8 = pd.concat([results8, pred], axis=1)

results8.shape

Epoch 1/25
820/820 [==============================] - 2s 2ms/step - loss: 1.4897 - val_loss: 1.6382
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4418 - val_loss: 1.6316
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4268 - val_loss: 1.6221
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4101 - val_loss: 1.6148
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4211 - val_loss: 1.5914
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4097 - val_loss: 1.5810
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4104 - val_loss: 1.5773
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4121 - val_loss: 1.5941
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4106 - val_loss: 1.5584
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4109 - val_loss: 1.5597

820/820 [==============================] - 2s 2ms/step - loss: 2.6669 - val_loss: 3.0763
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6435 - val_loss: 3.1570
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6621 - val_loss: 3.0707
Epoch 4/25
820/820 [==============================] - 1s 2ms/step - loss: 2.6358 - val_loss: 3.0786
Epoch 5/25
820/820 [==============================] - 1s 2ms/step - loss: 2.6569 - val_loss: 3.1156
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6334 - val_loss: 3.0470
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6663 - val_loss: 3.0826
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6568 - val_loss: 3.0765
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6574 - val_loss: 3.0836
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6602 - val_loss: 3.0690
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.6180 - val_loss: 3.0773
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6250 - val_loss: 3.1400
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6256 - val_loss: 3.1126
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6325 - val_loss: 3.1171
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5891 - val_loss: 3.0913
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6419 - val_loss: 3.0887
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6319 - val_loss: 3.1262
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6015 - val_loss: 3.0790
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6377 - val_loss: 3.0849
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5924 - val_loss: 3.0933
Epoch 12/

820/820 [==============================] - 1s 1ms/step - loss: 1.8945 - val_loss: 2.2556
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8833 - val_loss: 2.2862
Epoch 9/25
820/820 [==============================] - 1s 2ms/step - loss: 1.8647 - val_loss: 2.2267
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8969 - val_loss: 2.2510
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8518 - val_loss: 2.2704
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8682 - val_loss: 2.2873
Epoch 13/25
820/820 [==============================] - 2s 3ms/step - loss: 1.8788 - val_loss: 2.2987
Epoch 14/25
820/820 [==============================] - 2s 2ms/step - loss: 1.8761 - val_loss: 2.3241
Epoch 15/25
820/820 [==============================] - 2s 2ms/step - loss: 1.8642 - val_loss: 2.2699
Epoch 16/25
820/820 [==============================] - 1s 2ms/step - loss: 1.8530 - val_loss: 2.2895
Epoc

820/820 [==============================] - 1s 1ms/step - loss: 0.7408 - val_loss: 0.9373
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7380 - val_loss: 0.9306
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7438 - val_loss: 0.9215
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7374 - val_loss: 0.9012
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7317 - val_loss: 0.9318
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7482 - val_loss: 0.9170
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7238 - val_loss: 0.9441
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7388 - val_loss: 0.9336
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7347 - val_loss: 0.8986
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7430 - val_loss: 0.9059
Ep

(3888, 9)

In [ ]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results7.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results8.sort_index().values
submission

In [ ]:
submission.to_csv('./submission/submission_20210114-3.csv', index=False)